In [3]:
import numpy as np
import cv2

import matplotlib.pyplot as plt

import os

In [2]:
dataset_path = r'E:\Workspace\datasets\cats'

In [9]:
filenames_raw = []

for _, _, fnames in os.walk(os.path.join(dataset_path, 'raw')):
    for fname in fnames:
        if fname.split(os.path.extsep)[-1] == 'jpg':
            filenames_raw.append(fname)
    
    break
    
print(len(filenames_raw))

9993


In [41]:
for filename_raw in filenames_raw:
    img_raw = cv2.imread(os.path.join(dataset_path, 'raw', filename_raw))
    img_raw_data = []
    with open(os.path.join(dataset_path, 'raw', filename_raw + '.cat'), 'r') as datafile:
        img_raw_data = list(map(int, datafile.read().split()))
    
    if len(img_raw_data) < 5:
        continue
    
    p0 = np.array(img_raw_data[1:3], dtype=np.int32)
    p1 = np.array(img_raw_data[3:5], dtype=np.int32)
    dp = p1 - p0
    
    normal = np.array((0, 0), dtype=np.int32)
    normal[0] = dp[1]
    normal[1] = -dp[0]
    mid = (p0 + p1)/2

    c = 2

    rect = [
        mid + c*( - dp - normal),
        mid + c*( - dp + normal),
        mid + c*( + dp + normal),
        mid + c*( + dp - normal)
        ] 

    src = np.array(rect, dtype=np.float32)

    rect_side_length = (((rect[0] - rect[1])**2).sum())**.5

    out_size = 256

    dst = np.array([(0, out_size), (0, 0), (out_size, 0), (out_size, out_size)], dtype=np.float32)

    M = cv2.getPerspectiveTransform(src, dst)

    img_pocessed = cv2.warpPerspective(img_raw, M, (out_size, out_size))
    
    cv2.imwrite(os.path.join(dataset_path, 'train', filename_raw), img_pocessed)
